<img src="images/welcome.jpg">

### Primer día de trabajo

Descubrimos que nuestra primera tarea será construir un modelo que sea capaz de **predecir el precio de una propiedad en el Estado de California** a partir de un conjunto de datos relativos a los distintos distritos (población, renta promedio...); cada registro está etiquetado con el precio promedio de la propiedad en ese distrito. El objetivo es que nuestro modelo pueda predecir el precio medio a partir del resto de features aprendiendo de los datos disponibles.

### Reconociendo el escenario

A bote pronto, la primera pregunta obvia es consultar con el jefe **¿Cuál es el propósito final del proyecto?**; seguramente la construcción de este modelo, sea un paso más para obtener un beneficio de algo. Esto condicionará:
* El tipo de modelo que se escogerá.
* La métrica para evaluar la precisión del modelo.
* El tiempo dedicado a realizar ajustes sobre el mismo.

También el jefe nos informa de que el resultado de nuestro algoritmo irá a parar a su vez (junto con otros datos) a otro modelo que determinará si es ventajoso o no invertir en una determinada zona, con el consiguiente efecto en la rentabilidad obtenida por la compañía.

<img src="images/modelo.jpg">

Además es interesante saber **cómo se aborda el problema actualmente**, de este modo podremos tener una noción del rendimiento de la solución actual así como de algún insight adicional.

Hablando con el jefe, este nos comenta de que la tarea actualmente se realiza de forma manual mediante un conjunto de reglas elaboradas por un conjunto de expertos; se trata de un proceso costoso en tiempo y esfuerzo y con una tasa de error de alrededor del 15%.

### Coordinándonos

A tenor de lo que se puede ver en el flujo de datos de arriba, no somos los únicos en este proceso que discurre desde los datos en bruto hasta las decisiones de inversión. Por tanto es conveniente mantener una conversación previa con los equipos que van a depender de nuestros datos para asegurarnos de que entendemos sus necesidades.

Ciertamente podría darse el caso de que el sistema que reciba nuestros datos, esperara una clasificación categórica del tipo: *"Muy Barato"*, *"Barato"*, *"En la media"*, *"Caro"*, *"Muy Caro"* en vez de precios. Si esto fuera así, para nosotros no sería tan importante obtener el precio exacto.

Supongamos que después de hablar con el equipo responsable, nos confirman que ellos necesitarán precios y no categorías.

### ¿Qué tipo de tarea tenemos entre manos?,
* Aprendizaje ¿*Supervisado* ó *No Supervisado*?
* Si es Supervisado...¿Se trata de un problema de *Clasificación* o de *Regresión*?
* ¿Deberíamos prepararlo para se *entrenado online o no*?

# MANOS A LA OBRA

### Dataset

Está compuesto por datos del censo de California de los años 90. Cada registro representa un *Block Group* que es la unidad geográfica mínima para la cual el censo de Estados Unidos publica datos muestrales y que comprende una población entre 600 y 3000 personas. Podemos llamar al Block Group, distrito. [Más información](http://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html).

### Obteniendo los datos

En un entorno de trabajo, los datos estarían almacenados en alguna base de datos on premise o incluso en algún almacenamiento cloud (AWS, Azure, GCloud..); con lo que necesitaríamos unas credenciales de acceso así como familiarizarnos con el esquema de los datos. En este caso obtendremos los datos de Internet, (concretamente de [este](http://www.dcc.fc.up.pt/~ltorgo/) profesor de la Universidad de Porto) y elaboraremos una función para que, siempre que queramos, podamos obtener la última versión de los datos sin demasiadas manualidades (al final es también una excusa para ver unas cuantas librerías que nos pueden ser muy útiles en nuestro día a día).<br>
**NOTA IMPORTANTE**: En ocasiones, la página de recursos de la Universidad de Porto del profesor Luis Torgo, no está habilitada. Hemos subido el dataset al github de datahack school para asegurar su disponibilidad. No obstante todo el crédito del mismo es del profesor Luis Torgo.

In [ ]:
# Por cuestiones didácticas, iremos cargando las distintas
# bibliotecas y módulos, a medida que los vayamos necesitando.
# En la vida real, todas las importaciones irían al principio
# del código en orden alfabético (para facilitar su legibilidad).
import os
import tarfile
import requests

* **os** nos permitirá manejar y realizar diversas operaciones sobre nuestro sistema operativo: gestión de directorios y ficheros, ejecución de comandos, usuarios, permisos...
* **tarfile** como su propio nombre indica, proporciona diversas funcionalidades para gestionar el empaquetado de ficheros. También es capaz de escribir y leer ficheros comprimidos mediante gzip y bz2.
* **requests** ciertamente existen otras librerías capaz de manejar requests como `urllib` y `urllib2`; `requests` proporciona más funcionalidad y se ha convertido en el estándar de facto sobre como gestionar peticiones [http en Python](https://requests.readthedocs.io/en/master/).

In [ ]:
# URL donde se ubica nuestro fichero
#HOUSING_URL = "http://www.dcc.fc.up.pt/~ltorgo/Regression/"
HOUSING_GITHUB_URL = "https://github.com/datahack-school/resources/raw/master/"
# Nombre de nuestro fichero
HOUSING_TGZ_FILENAME = "cal_housing.tgz"
# Path relativo de nuestro portátil donde queremos que se descargue el fichero
HOUSING_LOCAL_PATH = "datasets/housing"

In [ ]:
def fetch_housing_data(housing_url, housing_file, housing_path):
    """
    Esta función permite obtener un determinado fichero .tar/.tar.gz/.targz ubicado en una
    determinada URL, guardarlo en un directorio local de nuestra elección y 
    desarchivarlo allí mismo.
    Argumentos:
        - housing_url: url donde nuestro fichero está alojado (omitiendo el nombre del fichero).
        - housing_file: nombre del fichero.
        - housing_path: ubicación local donde se quiere descargar y desarchivar el fichero.
    """

    # Comprueba que el directorio destino existe y si no créalo    
    if not os.path.isdir(housing_path):
        print(housing_path,"does not exist, it will be created...")
        os.makedirs(housing_path)
        print(housing_path,"created!")
    tgz_path = os.path.join(housing_path, housing_file)
    
    header = {'User-Agent': 'Mozilla/5.0'}
    # ¿Mozilla 5.0? para averiguar más sobre el User-agent https://webaim.org/blog/user-agent-string-history/
    print("requesting housing dataset at ",housing_url)
    # Hacemos una petición a la URL donde se ubican los datos
    # stream=True permite mantener la conexion abierta y descargar el contenido poco a poco
    response = requests.get(housing_url + housing_file, headers=header)
    
    # Si la request fue aceptada, volcaremos el contenido de la respuesta en nuestro disco (sobre los códigos
    # de respuesta de una petición HTTP https://en.wikipedia.org/wiki/List_of_HTTP_status_codes)
    if(response.status_code == 200):
        with open(tgz_path, 'wb') as handle:
            for block in response.iter_content(1024):
                handle.write(block)
            print("download complete!")
            
    # Lo siguiente es desempaquetar el contenido comprimido
    print("Untarring files...")
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    print("Extraction complete!")
    housing_tgz.close()    

Puesto que la ejecución de esta función trata de acceder a una ubicación remota para descargar el dataset, asegúrate de tener conexión a Internet antes de invocar a la función que hemos definido. Si ya habías ejecutado la descarga antes y quieres reproducir otra vez los pasos de la descarga para comprobar su funcionamiento, borra el subdirectorio *housing* del directorio *datasets* (que encontrarás en el mismo directorio de este notebook).

In [ ]:
fetch_housing_data(HOUSING_GITHUB_URL, HOUSING_TGZ_FILENAME, HOUSING_LOCAL_PATH)

Como resultado de la ejecución de la función anterior, el fichero *cal_housing.tgz* debería haberse descargado a un path relativo al directorio de nuestro notebook (*./datasets/housing*) y además haberse desarchivado y descomprimido, quedando como resultado dos ficheros: uno con las cabeceras (*cal_housing.domain*) y otro con los datos (*cal_housing.data*).

In [ ]:
# Directorio donde se ubican las dos partes (.data y .domain) de nuestro fichero
HOUSING_UNTAR_PATH = HOUSING_LOCAL_PATH + "/CaliforniaHousing"
# Nombre de nuestro fichero destino (resultante de combinar el .data y el .domain)
HOUSING_CONCAT_FILENAME = "housing.csv"
# Ruta completa donde se generará el fichero destino
HOUSING_TOTAL_PATH = HOUSING_LOCAL_PATH + "/" + HOUSING_CONCAT_FILENAME

Revisemos primeramente el fichero cal_housing.domain

In [ ]:
with open(HOUSING_UNTAR_PATH + "/cal_housing.domain") as header_file:
    print(header_file.read())

El fichero que contiene las cabeceras, no está precísamente formateado como a nosotros nos gustaría, es más, contiene un dato adicional además del nombre de la columna: su tipo. Como vemos todas las columnas son **"continuous"** lo cual nos indica que todas ellas tomarán valores numéricos.

A continuación, se imprimen las cinco primeras líneas del fichero de datos para tener una noción de su aspecto

In [ ]:
n_lines = 5
with open(HOUSING_UNTAR_PATH + "/cal_housing.data") as data_file:
    # arange es un método de numpy que devuelve un array de valores
    # que se corresponde con el intervalo [0,n_lines). Tanto el valor
    # inicial, como el final, como el salto entre valores..son configurables.
    for line in range(n_lines):
        print(next(data_file))

Concatenemos ahora ambos ficheros de manera que los nombres de los campos pasen a ser una única línea que constituya la cabecera del fichero de datos.

In [ ]:
import glob

* **glob** esta librería aporta una funcionalidad similar al [*ls*](https://www.tecmint.com/use-wildcards-to-match-filenames-in-linux/) de UNIX, permitiendo el uso de expresiones regulares con respecto a ubicaciones absolutas y relativas para obtener una lista de los ficheros que se correspondan con aquellas. Vamos a utilizar concretamente el método `glob` de la librería `glob`:

Nuestro objetivo ahora será obtener un único fichero a partir de los ficheros .domain y .data. Para ver el procedimiento, échale un vistazo a la diapositiva *00_project_Flow.ipynb (Concatenando ficheros)*.

In [ ]:
def merge_housing_data(housing_untar_path, housing_total_path):
    """
    Esta función barrerá el directorio donde el tar descargado fue extraído, 
    procesará el fichero con la cabecera (.domain) dejando solo los nombres de las
    features (sin su tipo) y lo volcará en un nuevo fichero seguido del contenido 
    del fichero que contiene el resto del total de los datos.
    Argumentos:
        - housing_untar_path: el directorio donde se encuentran los ficheros desempaquetados resultantes.
        - housing_total_path: la ruta del fichero concatenación de los dos resultantes.    
    """
    # glob.glob permitirá listar los ficheros contenidos en el directorio
    # sorted permitirá ordenar los ficheros obtenidos mediante glob.glob según el criterio
    # indicado, en este caso el tamaño (de menor a mayor tamaño).
    concat_files = sorted(glob.glob(housing_untar_path + "/*"), key=os.path.getsize)
    print("Merging files...",concat_files)
    with open(housing_total_path,"w") as outfile:
        for part_file in concat_files:
            with open(part_file, "r") as infile:
                # El fichero con los nombres de los campos será aplanado de manera que al final quede una
                # única línea: campo1, campo2, campo3, campo4...
                if(".domain" in part_file):
                    # Primero se leen todas sus líneas (el nombre de cada feature y su tipo)
                    raw_header = infile.readlines()
                    # Luego suprime el tipo (": continous") metiendo el resultado en una lista
                    # LIST COMPRENHENSION
                    header = [field.replace(": continuous.","").strip() for field in raw_header]
                    # Une cada elemento de la lista por "," y vuelca el resultado en el fichero de salida
                    outfile.write(",".join(header)+"\n")
                else:
                    outfile.write(infile.read())   
    print("Files merged into",outfile.name)

In [ ]:
merge_housing_data(HOUSING_UNTAR_PATH, HOUSING_TOTAL_PATH)

¡Ya tenemos nuestro fichero listo para comenzar a trabajar sobre el!

### Toma de contacto con los datos

Cargaremos el dataset en una de las estructuras de datos más versátiles que Python (y otros lenguajes como R) ofrece para Machine Learning: el **DataFrame** que, básicamente, es una estructura tabular constituída por columnas que pueden contener datos de diversos tipos.

In [ ]:
import pandas as pd

In [ ]:
print("Pandas version " + pd.__version__)

* **pandas** es otra librería que, si llegamos a manejar con soltura, nos ayudará a ser unos científicos de datos de lo más eficientes en Python. El DataFrame de `pandas` es la estructura tabular por excelencia que Python ofrece para el Machine Learning, además provee todo un arsenal de funciones que permiten procesarlo, rebanarlo (slicing), modificarlo y manipularlo de distintas formas.

Hagamos una sencilla función que nos permite cargar los datos en un DataFrame de `pandas` (sí, sí...para dos líneas es tontería hacer un función, pero también es bueno acostumbrarnos a verlas...y a escribirlas :-) )

In [ ]:
def load_housing_data(housing_path, housing_filename):
    """
    Esta función concatena el path del directorio donde se encuentra el fichero 
    total con el nombre de este formando así el path absoluto. Devuelve 
    el contenido del fichero como un DataFrame de pandas.
    Argumentos:
        - housing_path: el directorio donde el fichero total debe de estar alojado.
        - housing_filename: el nombre del fichero total, con sus dos partes.
    """
    csv_path = os.path.join(housing_path, housing_filename)
    return pd.read_csv(csv_path)

**¡ALTO!** antes de cargar los datos en un DataFrame, duplicaremos manualmente algunas de las primeras líneas para ver su tratamiento posterior (abriremos **housing.csv** con algún editor de texto tipo [Notepad++](https://notepad-plus-plus.org/downloads/v7.8.4/) y duplicaremos alguna línea. No se debe utilizar Excel ya que puede hacer que el formato columnar se descuadre).

In [ ]:
# Si ya has "tocado" los datos a tu gusto...¡dale caña!
housing = load_housing_data(HOUSING_LOCAL_PATH, HOUSING_CONCAT_FILENAME)

Con ayuda de la función `read_csv()`, por fín tenemos nuestros datos cargados en un DataFrame de `pandas`; la función `head()` de esta librería, permite obtener de  manera sencilla una visión de como es nuestro fichero (acordaos del "lío" que era hacer lo mismo directamente desde el fichero. Los DataFrame de `pandas` facilitan bastante la vida.

In [ ]:
housing.head(5)

In [ ]:
type(housing)

Igualmente la función `info()` nos muestra un resumen conciso de las características del DataFrame (número de columnas, nombre de las mismas, número de elementos no nulos, tipología de los mismos...). Vemos que disponemos de un dataset con, aproximadamente, 20640 elementos donde cada entrada es un distrito (cierto que el tamaño del dataset es reducido para un caso real, pero es adecuado para foguearse).

In [ ]:
housing.info()

### Revisando duplicados

Antes de profundizar mucho en nuestro dataset, es conveniente cerciorarnos de que no haya duplicados en el. Para ello tenemos varias instrucciones que nos pueden servir de ayuda: `duplicated()` por ejemplo marca como `True` todas aquellas filas (registros) que tienen el mismo valor en todos sus campos (marcandolas como `False` en caso contrario). Si quisiéramos solo centrarnos en algún campo en concreto le pasaríamos a `duplicated` una lista con los nombres de los campos a tomar en consideración:  `duplicated('campo1')` ó `duplicated(['campo1','campo2'...])`. **¡Ojo!** esta instrucción no elimina duplicados, ni modifica el DataFrame, solo los muestra.

In [ ]:
# Ojo al parámetro keep que determina qué registros son marcados como True (duplicados)
# Si keep vale 'first' todos los registros duplicados salvo la primera ocurrencia son marcados como True.
# Si keep vale 'last' todos los registros duplicados salvo la última ocurrencia son marcados como True.
# Si keep vale False todas las ocurrencias de los registros duplicados son marcadas como True (ojo, False es un 
# tipo booleano, no va entrecomillado)
housing[housing.duplicated(keep='first')]

Lo siguiente sería eliminar los duplicados detectados, para ello utilizaríamos la sentencia `drop_duplicates()` indicando mediante el parámetro `keep` si queremos preservar la primera aparición (`'first'`, comportamiento por defecto), la última (`'last'`) ó eliminar todo duplicado (`False`). Además, el parámetro `inplace` permitirá que la eliminación de duplicados se haga in situ, sobre el propio DataFrame sin necesidad de asignar el resultado de la función a un nuevo DataFrame. Este parámetro es característico de todo método de `pandas` que conlleve posibles modificaciones del DataFrame.

In [ ]:
housing.drop_duplicates(inplace=True)

In [ ]:
housing.head(10)

Vemos que al revisar de nuevo los primeros registros, hay huecos en el label del indexado...esto en si mismo no es un problema, pero hay que tener en cuenta que si se realizan manipulaciones sobre los datos que se apoyen en ese label, se pueden obtener resultados inesperados. Podemos resetear el label antes de que seguir realizando manipulaciones sobre los datos mediante `reset_index()`, que nos permitirá reiniciar el índice de manera que no haya huecos. Si esto del label y el index te suena algo raro, échale un vistazo a la diapositiva *00_project_Flow.ipynb (dataframes)*

In [ ]:
housing.reset_index(drop=True, inplace=True)

Comprobamos que los índices han recuperado su estado original

In [ ]:
housing.head(10)

### Pickling time!!
<img src="images/pickle.jpg">

Imaginad que habéis empleado una tiempo importante en una determinada tarea que implica cargar una cantidad de datos en un objeto Python: desde scrapear datos de miles de webs, calcular millones de dígitos de pi o simplemente hacer cambios en un DataFrame pesado.

Si la batería del portátil se termina de forma traicionera o simplemente la sesión de Python muere, esas estructuras de datos que tanto han costado generar se perderán (a veces la gente se pregunta "pero cuando abro el notebook sigo viendo las trazas de ejecución...", efectívamente, eso es todo lo que queda de tu ejecución anterior: las trazas :-) ).
    
`pickle` permite guardar un objeto Python (DataFrame de `pandas`, array de `numpy`, diccionario...) como un fichero binario en tu disco duro. Una vez guardado, ya da igual que tu sesión muera o que reinicies tu máquina. El objeto estará ahí disponible para que lo cargues cuando lo necesites.

Probemos a guardar el DataFrame en `pickle`:

In [ ]:
# Si existe otro pickle con el mismo nombre, se sobreescribirá
housing.to_pickle('california.guau')

In [ ]:
# Borremos el DataFrame para hacer la demostración de cómo se vuelve a cargar
del housing

In [ ]:
# Restauramos la variable a partir del pickle
housing = pd.read_pickle('california.guau')

In [ ]:
housing.head(3)

¿Y si no estamos trabajando con `pandas`?

In [ ]:
import pickle

In [ ]:
#Probemos con un diccionario
dictionary = {'x':[3,1,2,3], 'y':[9,2,1,3], 'status':[True, False], 'res':'A123'}

In [ ]:
# Guardamos
with open('dict.pickle', 'wb') as f:
    pickle.dump(dictionary, f)

In [ ]:
# Borramos el diccionario para asegurarnos de
# que no exista ningún objeto con ese nombre cuando lo creemos
del dictionary

In [ ]:
# Cargamos
with open('dict.pickle', 'rb') as f:
    dictionary = pickle.load(f)

In [ ]:
dictionary

#### Intercambiando pickles

Un objeto guardado mediante `pickle` puede ser compartido por correo, en un drive, en un USB...¡donde sea! Toda esta flexibilidad tiene el inconveniente de que cierta gente puede preparar pickles maliciosos que ejecuten algún tipo de código malicioso en tu máquina. Así que es conveniente solo cargar aquellos pickles cuya fuente se conozca.

### Haciendo nuestro dataset algo más interesante

Nuestro dataset es bastante aburrido, **no tiene ni un valor nulo ni tampoco features categóricas**...vamos a intentar retocar algunos de los datos para darle un poquito de emoción a esto.
Vamos a crearnos una función que aplique una máscara a una de nuestras features de manera que un determinado % de sus valores se vuelva nulo preservando el resto. Para más información sobre el método `mask`, echa un vistazo a la diapositiva *00_project_Flow.ipynb (mask)*.

In [ ]:
import numpy as np

In [ ]:
print("Numpy version " + np.__version__)

* **numpy** librería fundamental para cualquier científico de datos que opte por desarrollar su actividad en Python; `numpy` es *LA LIBRERÍA* para manipular vectores y estructuras matriciales de cualquier dimensión.

In [ ]:
def set_random_null_by_column(column, pandas_dataframe, null_pctge=.15):
    """
    Esta función tiene por objeto establecer a NaN un determinado
    porcentaje de los valores de la columna que se indique.
    Argumentos:
        - column: el nombre de la columna que se quiere nulificar.
        - pandas_dataframe: el dataframe de pandas que queremos modificar.
        - null_pctge: porcentaje de valores de la columna que deben de ser modificados.
    """
    # shape es una propiedad de todos objeto DataFrame de pandas
    col_shp = pandas_dataframe[column].shape
    # Creamos una máscara con dos posibles valores True/False
    mascara_nan = np.random.choice([True, False], size=col_shp, p=[null_pctge,1-null_pctge])    
    # Al aplicar la máscara al DataFrame, ojo, True reemplazará el valor del campo elegido por NaN y False lo preservará
    pandas_dataframe[column] = pandas_dataframe[column].mask(mascara_nan)
    return pandas_dataframe

Elegiremos por ejemplo la feature totalBedrooms

In [ ]:
housing = set_random_null_by_column("totalBedrooms", housing, null_pctge=.25)

In [ ]:
housing.info()

Bueno...vemos que uno de nuestros campos ya tiene unos cuantos valores NaN lo cual hará que nos tengamos que encargar de hacer algo con los datos antes de alimentar un modelo con ellos. Pero no es suficiente destrozo; vivimos en un mundo de categorías y como tal **no es concebible un dataset sin variables categóricas**, así que vamos a insertar una que asigne la proximidad al oceano de cada distrito. Empezaremos definiendo los valores de dicha categoría y la probabilidad de que aparezcan en el dataset.

In [ ]:
ocean_proximity_cat = ['<1H OCEAN', 'INLAND', 'NEAR OCEAN', 'NEAR BAY' ,'ISLAND']
# Esto de abajo son probabilidades...sí, parecen raras pero suman 1
ocean_proximity_prob = [.44, .32, .12, .11 , .01]

Crearemos un array de `numpy` en el que se distribuyan las categorías según las probabilidades indicadas.

In [ ]:
ocean_prox_values = np.random.choice(ocean_proximity_cat, size=(housing.shape[0],1), p=ocean_proximity_prob)

In [ ]:
type(ocean_prox_values)

La creación de una nueva columna en nuestro DataFrame es tan fácil como referenciarla como si ya existiera y asignarle sus valores mediante un array de `numpy`

In [ ]:
housing["ocean_proximity"] = ocean_prox_values

In [ ]:
housing.info()

Y ya tenemos nuestra nueva columna categórica; podemos comprobar cómo se han distribuído las categorías entre los distritos mediante la función `value_counts()`. Normalmente aquellos valores que se repiten con frecuencia en un dataset, tienen altas probabilidades de corresponder a una feature categórica.

In [ ]:
housing["ocean_proximity"].value_counts()

### Revisando nuestro dataset

Vamos a ver dos maneras de abordar la exploración de nuestro dataset, primero una más numérica y luego otra más gráfica. Para la primera de ellas, la función `describe()` permite obtener un resumen estadístico de la distribución de nuestras features numéricas.

In [ ]:
housing.describe()

Aquí tenemos información que puede ser interesante (ojo a cómo afecta la presencia de NaN a cada estadístico),
* **count** Contabiliza todos aquellos valores que no son NaN.
* **mean** Calcula la media de los valores para cada feature (de nuevo, se ignoran los valores NaN).
* **min** Obtiene el mínimo valor para cada feature.
* **max** Obtiene el máximo valor para cada feature.
* **std** Muestra la desviación estándar proporcionando una medida de la dispersión de los valores con respecto a la media (mean), cuanto mayor sea la desviación estándar, menos representativa será la media de la distribución.
* **25%**,**50%**,**75%** Los cuartiles representan el valor por debajo del cual se situan un determinado porcentaje de los valores para una determinada feature. Por ejemplo el primer cuartil de la feature population nos indica que en el 25% de los distritos viven menos de 787 personas.


In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
print("Matplotlib version " + matplotlib.__version__)

In [ ]:
housing.hist(bins=50, figsize=(20,15))
plt.show()

#### ¿Algo raro en el medianIncome, housingMedianAge y medianHouseValue?¿Algo que resaltar sobre las colas de las distribuciones?

1. La feature **medianIncome** que, en principio, parecería lógico que estuviera expresada en USD, no parece estarlo. Consultando con el equipo encargado de recolectar los datos, nos dicen que han hecho un escalado de los datos de manera que el valor máximo es ahora 15 y el valor mínimo 0.5 Es normal encontrarse con escalados de este tipo y no es algo necesariamente malo, pero sí que es conveniente conocer la lógica que se ha seguido.

2. Las features **housingMedianAge** y **medianHouseValue** tienen un pico muy sospechoso en sus valores más alto. El problema es que la segunda de ellas es nuestra label o variable target y si los datos se han dispuesto de tal manera que todos aquellos valores de medianHouseValue mayores de 500000 se reflejarán como 500000, nuestro modelo puede aprender que es el tope con el que puede valorar una propiedad. Aquí tenemos dos opciones,
    * Que para los distritos afectados por esto, se recolecten los valores verdaderos.
    * Eliminar todos los distritos de nuestros datos cuyo medianHouseValue ha sido limitado (tanto del training como del test set) *housing = housing[housing.medianHouseValue < 500000]*
    
3. Parece que predominantemente tenemos distribuciones asimétricas hacia la derecha, esto es, la media caerá a la derecha de la mediana. Esto puede hacer que nuestros modelos tengan más problemas a la hora de detectar patrones que si la distribución fuera más simétrica.

<img src="images/asymmetry.png">

### Muerte por corchetes
¿Existe alguna manera un poco más simple y legible de indexar un DataFrame que no sea apelotonando corchetes y paréntesis? Sí, se puede transformar esto:

In [ ]:
housing[((housing['medianHouseValue'] > 500000) & (housing['totalBedrooms'].notnull())) | (housing['ocean_proximity'] == 'ISLAND')]

en esto:

In [ ]:
housing.query('(medianHouseValue > 500000 and not totalBedrooms.isnull()) or (ocean_proximity == "ISLAND")', engine='python')

Para aquellas personas de bien que les sangren los ojos con el hardcoding, existe también la posibilidad de referenciar variables de nuestro entorno en la query:

In [ ]:
# Nuestra variable
carisimo = 500000

In [ ]:
# La incorporamos a la query precediéndola de @
housing.query('(medianHouseValue > @carisimo and not totalBedrooms.isnull()) | (ocean_proximity == "ISLAND")',engine='python')

In [ ]:
# Si alguna persona no se siente confortable con el número de filas mostrado por pantalla
# se puede modificar mediante la siguiente opción
pd.set_option('display.max_rows',10)
# En este caso el número de columnas no es un problema...pero si lo fuera, la siguiente opción
# permite ajustar dicho número
pd.set_option('display.max_columns', None)

Lo mejor de todo, es que independientemente de la legibilidad, `query()` nos permite además optimizar el uso de la memoria. Cualquier expresión compuesta, conlleva por debajo la creación de arrays temporales auxiliares. Esto en principio no es un problema a no se que el tamaño de nuestro array o nuestro DataFrame (normalmente estos son más propensos a crecer desmesuradamente que los arrays) se incremente en exceso, pero es conveniente tenerlo en cuenta.

El parámetro`engine`, puede plantear incógnitas ya que no es algo que esté ampliamente documentado. Para el método `query()`, engine puede tomar dos valores:
 * `numexpr`: el valor por defecto, en la mayoría de las casuísticas es la elección más adecuada en términos de rendimiento.
 * `python`: la opción que utilizaremos si queremos meter en nuestras "queries" funciones de Python, por lo general su rendimiento es inferior a `numexpr`, tan solo en el tramo de datasets que rondan los 15K-20K registros y tres o cuatro features, llega a mejorar a `numexpr`

### Separación datos en conjuntos de train y test

En este punto nos despediremos de una parte de nuestros datos hasta el final del proceso: nunca los utilizaremos, los miraremos o pensaremos en ellos hasta que toque evaluar el error de generalización del modelo elegido.

Si tomamos en consideración los datos de test a la hora de escoger nuestro modelo, cuando llegue el momento de obtener el error de generalización nos encontraremos con que el desempeño del modelo es espectacular PERO al ponerlo en Producción nos daremos de bruces con la cruda realidad y es que, muy probablemente, nuestro modelo no tendrá un performance tan bueno como creíamos (sobreajuste u overfitting).

* **Data Snooping Bias** una de las consecuencias posibles de no respetar los datos de test y utilizarlos también para detectar patrones sobre ellos; es la posibilidad de que algunos de los patrones detectados sean puramente aleatorios y que no se cumplan en los nuevos datos a los que nuestro modelo tenga que enfrentarse de modo que este tendrá un pobre desempeño ante nuevos datos. Una manera de evitar esto es, justamente, el uso de un conjunto de test completamente desconocido para nuestro modelo sobre el que probará si realmente es un modelo balanceado y capaz de generalizar o peca de sobreajuste.

`scikit-learn` proporciona varios métodos para separar datasets en múltiples subconjuntos de distintas maneras (por ejemplo, `train_test_split`). Estos métodos, tienen como particularidad que pueden recibir como parámetro una semilla de manera que, aunque los ejecutemos múltiples veces, la separación resultante será la misma que la primera vez; esto es algo muy interesante ya que de no ser así, estaríamos incurriendo en el **data snooping bias**. 
Además puede recibir múltiples datasets siempre y cuando tengan idéntico número de filas, la utilidad de esto es que si por ejemplo nos pasan las labels en otro DataFrame diferente a las features, garantizamos que la división entre train set y test set será idéntica en ambos DataFrame.
    
    

Pero...**¿basta con introducir aleatoriedad en el particionamiento?** es posible que si el dataset es muy grande sí; pero existe el riesgo de introducir **sampling bias**. Si tenemos una población que está dividida en grupos homogéneos o estratos, es necesario que en nuestros subconjuntos de training y test, se mantenga la proporción entre dichos estratos. Para más información sobre esto, echa un vistazo a la diapositiva *00_project_Flow.ipynb (sampling bias)*.

<img src="images/digest.png">

Supongamos que hemos hablado con los expertos que se encargan actualmente de realizar los cálculos y nos han dicho que para predecir el precio medio de la propiedad, **es muy importante la feature medianIncome**, es decir la renta promedio; por tanto nos interesa que los distintos "subgrupos" de medianIncome estén proporcionalmente representados en nuestros conjuntos de training y test...pero hay otro problema ¿cómo obtenemos los subgrupos de una feature con valores numéricos continuos?. Vamos a ello

In [ ]:
# Empezamos visualizando medianIncome
housing.hist(column="medianIncome", bins=50, figsize=(4,4))
plt.title("medianIncome")
plt.show()

La integración de `matplotlib` con `pandas`, nos permite obtener visualizaciones rápidas y eficaces. Aunque puede ser que a veces busquemos algo más interactivo. Una manera relativamente asequible de conseguir esta interactividad es `hvplot`. [Aquí](https://hvplot.holoviz.org/user_guide/Pandas_API.html) tenéis una referencia a la API de `pandas`. **Recordad que para que os funcione el siguiente import, previamente tendréis que haber instalado la librería del modo que se indicaba en el LEEDME.docs**

In [ ]:
import hvplot.pandas

In [ ]:
housing.hvplot.hist(y="medianIncome", bins=50, width=400)

Examinando el histograma para medianIncome, vemos que la mayoría de los valores de dicha feature, se agrupan en torno a 1.5 y 6, aunque hay algunos que se alejan bastante de 6. Los estratos que conformemos tienen que estar lo suficientemente bien representados. Esto se traduce en no volvernos locos definiendo demasiados estratos y que, por tanto, cada estrato sea lo suficientemente grande.

In [ ]:
# Vamos a discretizar los valores de medianIncome.
# Optaremos por definir cinco estratos que etiquetaremos como
# 1, 2, 3, 4 y 5. 
# El primer estrato irá de 0 a 1.5
# El segundo de 1.5 a 3
# El tercero de 3 a 4.5
# El cuatro de 4.5 a 6
# El quinto y último de 6 en adelante
# Realmente esto no es algo escrito en piedra, podéis jugar con estos rangos
# y comprobar si los estratos obtenidos son adecuados de acuerdo a los consejos 
# que hemos visto.
housing['incomeCat'] = pd.cut(housing['medianIncome'], bins=[0, 1.5, 3, 4.5, 6, np.inf], labels=[1,2,3,4,5])

In [ ]:
# Hemos creado una nueva feature llamada incomeCat que 
# es una versión discretizada de medianIncome.
# Podéis echar un ojo a los 10 primeros registros para ver
# que los valores de medianIncome han ido a parar al estrato esperado
housing[['medianIncome', 'incomeCat']].head(10)

Algo tan simple como pudiera parecer un histograma de una feature categórica, no es trivial de conseguir con `matplotlib`. Lo bueno que existen otras opciones para lograrlo sin demasiado esfuerzo. Como por ejemplo `seaborn` que pone una capa de abstracción sobre `matplotlib` lo cual consigue que la interacción con esta sea algo más amigable.

In [ ]:
import seaborn as sns

In [ ]:
print("Seaborn version " + sns.__version__)

In [ ]:
sns.countplot(housing['incomeCat'], color='blue')

Ahora se hará el muestreo estratificado en base a la categoría sintética que hemos creado, lo haremos utilizando el parámetro `stratify` de `train_test_split`

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
strat_train_set, strat_test_set = train_test_split(housing, test_size=0.2, random_state=42, stratify=housing['incomeCat'])

Veamos cual era la proporción inicial de las categorías ó estratos resultantes de incomeCat para el dataset completo

In [ ]:
housing["incomeCat"].value_counts() / len(housing)

Y ahora comprobemos que las proporciones se corresponden con los conjuntos de training y test resultantes

In [ ]:
# Conjunto ó dataset de entrenamiento (training)
strat_train_set["incomeCat"].value_counts() / len(strat_train_set)

In [ ]:
# Conjunto ó dataset de prueba (test)
strat_test_set["incomeCat"].value_counts() / len(strat_test_set)

Como vemos el stratified sampling nos ha servido para mantener en nuestros conjuntos de train y test la misma proporción de categorías de incomeCat que en el conjunto inicial. Ahora ya podemos prescindir de esta feature auxiliar que, no olvidemos, habíamos creado a partir de medianIncome.

In [ ]:
for data_set in [strat_train_set, strat_test_set, housing]:
    data_set.drop(["incomeCat"], axis=1, inplace=True)

Es posible que hayáis obtenido un maravilloso warning del paso anterior. Un warning es lo suficientemente importante para que investiguemos su causa.
El caso de SettingWithCopyWarning, tiene por objeto poner de manifiesto, aquellas operaciones con DataFrames para cuyo resultado no siempre está claro si lo que se obtiene es una vista del DataFrame (*view*) o una copia (*copy*). Esto es grave porque:
* Modificar una vista de una DataFrame implicaría modificar el DataFrame original.
* Modificar una copia de un DataFrame, no implica modificar el original.


Este warning parece decirnos que no está claro si `train_test_split` devuelve copias de nuestro DataFrame original o vistas. Navegando un poco nos damos cuenta de que en este caso es debido a una especie de descoordinación entre `scikit-learn` y `pandas`:
https://github.com/scikit-learn/scikit-learn/issues/8723

Dicho lo cual, en este caso el warning no tiene mayor influencia en nuestro código.

In [ ]:
strat_train_set.head()

Ha costado pero de momento hemos dejado fino el tema del particionamiento de datos en training y set. Podemos avanzar.

### Exploración de datos

Hasta el momento la toma de contacto con los datos no ha sido demasiado profunda, aunque ya hemos hecho algunas cosas. Para los siguientes pasos nos aseguraremos de,
1. El conjunto de test está apartado y aislado, nos olvidaremos de el por completo.
2. La exploración de datos se realizará solo en base al training set y si este es muy grande, obtendremos de el un set de exploración para que esta sea rápida y eficiente.

En nuestro caso el conjunto de training es manejable, así que directamente nos haremos una copia del mismo para no cargárnoslo en alguna manipulación. ¡Ojo! el método `copy()` es bastante importante ya que en principio podemos pensar que con una simple asignación a través de `=`, sería suficiente para crearnos un nuevo DataFrame...si hacemos esto lo que realmente estaremos creando es una referencia al DataFrame original, de manera que si modificamos la referencia, el original también se verá afectado (y viceversa). Con el método `copy()`, manteniendo el valor de su único parámetro `deep` a `True` (por defecto es así), se consigue un copia completa de manera que tengamos dos DataFrame independientes. ¿Copias y referencias? si te suena raro, échale un vistazo a la diapositiva *00_project_Flow.ipynb (Sabiendo asignar)*

In [ ]:
# housing será un DataFrame obtenido mediante el método copy()
housing = strat_train_set.copy()

In [ ]:
# housing2 será una referencia al DataFrame original
housing2 = strat_train_set

In [ ]:
# Cojamos un label al azar de nuestro DataFrame, por ejemplo: 17606
# Veamos el valor de strat_train_set para una determinada feature
# y el índice escogido
strat_train_set.loc[17606, 'medianIncome']

In [ ]:
# Y también para la referencia...
housing2.loc[17606, 'medianIncome']

In [ ]:
# Cambiemos el valor en la referencia
housing2.loc[17606, 'medianIncome'] = 7

In [ ]:
# Por ser una referencia...afecta también al DataFrame original
strat_train_set.loc[17606, 'medianIncome']

#### Visualizando de forma fácil e informativa

Puesto que tenemos entre las features latitud y longitud (latitude y longitude en el dataset) vamos a ver que ocurre si disponemos los registros que tenemos en base a sus coordenadas (al final es una excusa para introducir otro método de `pandas`: el método `plot()`, que invoca por debajo a `matplotlib` y que permite crear sobre nuestro DataFrame, distintos tipos de visualizaciones: líneas, barras, tartas, histogramas...). Probemos con un gráfico de dispersión.

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude")

Bueno...no nos han engañado, se parece bastante a California...pero tampoco es que nos aporte mucho. El método `plot` es muy potente pero a cambio, para sacarle provecho, requiere de nuestra parte no solo que consultemos la documentación relativa al método, sino que además tengamos en cuenta aquellos parámetros de `matplotlib` propios de la visualización que queremos plasmar.
Por ejemplo, en el caso del gráfico de dispersión (scatter plot) existe un parámetro llamado **alpha** que permite destacar aquellas zonas con mayor densidad de puntos.

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)

¡Aquí ya tenemos algo más de información! Vemos una mayor densidad en las zonas correspondientes con las grandes áreas urbanas de California: Los Angeles, San Francisco, San Diego, Fresno, Sacramento...Pero aun podemos meter más contenido en el mapa, para ello utilizaremos dos propiedades del tipo de plot que estamos visualizando (un **scatterplot**): el color y el tamaño (size).

- **c** representa el color. Permite especificar la feature cuyo valor determinará el color del punto y que se traducirá en un color determinado por:
- **cmap** : el [mapa de colores](https://matplotlib.org/stable/tutorials/colors/colormaps.html) seleccionado.
- **s** el tamaño de cada punto vendrá determinado por el valor de la feature que asignemos a este parámetro.

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4, s=housing["population"]/100, label="population",
            c="medianHouseValue", cmap=plt.get_cmap("autumn_r"), colorbar=True)

Aquí ya se puede ver más claramente que, en apariencia, los precios más altos coinciden con grandes núcleos urbanos cercanos a la costa. De nuevo, hay que tener en cuenta que el obtener una visualización adecuada no es algo trivial, sino que requiere de probaturas y de darle varias vueltas: por ejemplo algo tan simple como dividir population entre 100 para evitar que el tamaño de los puntos se nos vaya de las manos, ó encontrar una paleta de color adecuada para cmap...

Y la versión con `hvplot`...

In [ ]:
from bokeh.models.formatters import BasicTickFormatter

In [ ]:
housing.hvplot.scatter(x="longitude", y="latitude", 
                       alpha=0.4, s=housing["population"]/100, 
                       label="population", c="medianHouseValue", 
                       cmap=plt.get_cmap("autumn_r"), colorbar=True).opts(plot=dict(colorbar_opts={'formatter': BasicTickFormatter(use_scientific=False)}))

### Outliers

<img src="images/outlier.png">

#### Definición
Se entiende por outlier, aquel valor que es distante numéricamente del grueso de la distribución a la que pertenece.
El origen de un outlier puede estar en errores humanos o mecánicos en su captura o puede que sean valores auténticos, pero extremos en el dataset.
Incluso su presencia puede ser indicativo de algo. Por ejemplo en un dataset de transacciones, puede ayudar a identificar movimientos fraudulentos.

La definición de outlier es tan sencilla como vaga. Y es que en última instancia lo que es un outlier y lo que no, es un ejercicio algo subjetivo y que depende en buena parte del caso de negocio al que refiera el problema.

* ¿Cómo de outlier es un outlier? No siempre es ideal quitar outliers que *no lo son mucho*.
* ¿Qué ocurre si nuestro dataset tiene un gran número de features? ¿Para cuántas de ellas una observación tiene que contener outliers de manera que decidamos prescindir de ella?

En la práctica en problemas con muchas features es poco común atreverse a quitar outliers por algún lado. Hay modelos además que son más sensibles a outliers (como por ejemplo la regresión logística y los SVM) y otros que los suelen asimilar sin mucho problema (como los árboles de decisión o K-nearest neighbours).

#### Detección
Algunos métodos para la detección de outliers:
##### Análisis de valores extremos (Extreme Value Analysis)
El objetivo de este método es identificar las colas de las distribuciones de las features y encontrar los valores ubicados en sus extremos. En una distribución gaussiana, los outliers cumplirán alguna de estas características:
* Mayores que la media + 3 veces la desviación típica.
* Menores que la media - 3 veces la desviación típica.

Segúramente suene familiar la [**regla 68–95–99.7**](https://en.wikipedia.org/wiki/68–95–99.7_rule). Básicamente esta regla nos dice que para una distribución normal el 68% de los valores se encuentran a una desviación típica de la media, el 95% a dos desviaciones típicas y el 99.7% a tres.

En el siguiente apartado (Z-score), veremos un método rápido de obtener los outliers en base a este criterio, aunque siempre es posible calcularlo de manera "manual" obteniendo la media y la desviación típica para una feature mediante los métodos `mean()` y `std()`.

Centrémonos ahora en el caso más habitual...*¿qué pasa si nuestra distribución NO es normal?*.
En este caso se puede recurrir al **rango intercuartílico**:
<img src="images/IQR.png">
* Primero se calcula dicho rango: IQR = cuantil 75 (tercer cuartil) - cuantil 25 (primer cuartil)
* Límite superior = cuantil 75 (tercer cuartil) + (IQR * 1.5)
* Límite inferior = cuantil 25 (primer cuartil) - (IQR * 1.5)
Es posible que el 1.5 haya que reemplazarlo por 3 para casos más extremos.

Si tomamos de nuevo como ejemplo medianIncome:

In [ ]:
housing = strat_train_set.copy()

In [ ]:
# Calculamos el rango intercuartílico
IQR = housing["medianIncome"].quantile(0.75) - housing["medianIncome"].quantile(0.25)

In [ ]:
# Calculamos el límite superior normal y extremo (en el caso del inferior la operativa
# sería muy parecida)
limSup = housing["medianIncome"].quantile(0.75) + (IQR *  1.5)
limSupExt = housing["medianIncome"].quantile(0.75) + (IQR *  3)

Vamos a ver la proporción de observaciones que se encuentra por encima de estos límites:

In [ ]:
# Total de observaciones
total = housing['medianIncome'].shape[0]

In [ ]:
# Examinamos el límite superior calculando la proporción
len(housing[housing['medianIncome'] > limSup])/total

Esto quiere decir que un 3% de las observaciones están por encima del límite superior.

In [ ]:
# Examinamos el límite superior extremo
housing[housing['medianIncome'] > limSupExt].shape[0]/total

Tan solo un 0.6% de las observaciones están por encima del límite superior extremo. Se procedería de forma análoga para revisar aquellos valores situados a la izquierda de la distribución.

##### Z-Score
Es una medida de la distancia, en términos de desviaciones estándar, a la que una observación se encuentra de la media. Esta medida puede ser positiva (indicando que la observación se encuentra por encima de la media) o negativa (indicando que la observación se encuentra por debajo de la media).

Una de las ventajas del Z-score es que permite eliminar los efectos de la escala de los datos (es decir, da igual que los datos se muevan entre 0 y 13 que entre -2823020.2 y 49330202222.3) esto permite comparar features y datasets diferentes.

In [ ]:
from scipy import stats

In [ ]:
header = list(housing)

In [ ]:
# Obtenemos los z-score para cada feature
for feat in header:
    try:
        z = stats.zscore(housing[feat])
        print("feature: " + feat + ". Max z-score: " + str(z.max()) + ". Min z-score: " + str(z.min()))
    except TypeError:
        print("Can't calculate z-score for feature " + feat + ". It's "+ str(type(housing[feat].values[0])))
        pass

In [ ]:
# Tendríamos unos cuantos extremos que investigar, por poner un ejemplo
# vamos a echar un vistazo a la distribución de la feature population que se va 
# más allá de 30...
z = stats.zscore(housing['population'])

In [ ]:
# Examinemos pues aquellos z-score mayores que 30
print(np.where(z > 30))

In [ ]:
# ¿A qué registros corresponden esos z-score? lo sabremos a partir de los índices devueltos por numpy where
housing.iloc[np.where(z > 30)]

In [ ]:
# Un gráfico de dispersión a veces nos da perspectiva de nuestros posibles outliers
plt.scatter(np.arange(len(housing)), housing['population'], c=z>10, cmap=plt.get_cmap("winter"), alpha=0.5)

Vamos a rizar un poco más el rizo...

##### Clustering
<img src="images/kmeans.gif">

Cortesía de [Jeremy Jordan](https://www.jeremyjordan.me/grouping-data-points-with-k-means-clustering/)

El objetivo de las técnicas de Clustering, es agrupar en clusters o conjuntos, aquellas observaciones más similares entre sí (el concepto de similitud puede vernir dado por una métrica del tipo distancia Euclídea por ejemplo). Los pasos básicos de un algoritmo típico de clustering como el K-means son:
* Determinar el número de clusters que se quiere obtener (`k`).
* Asignar las `k` primeras observaciones como centroides provisionales de los `k` clusters.
* Calcular la distancia escogida (por ejemplo la Euclídea) de cada observación con respecto a cada centroide y en función del resultado asignar cada observación al cluster correspondiente al centroide más próximo.
* Después de haber completado los pasos anteriores, recalcular cada centroide en base a la media de todas las observaciones asignadas a su cluster.
* Repetir el proceso de cálculo de distancias, asignación de observaciones y recálculo de centroides hasta que no haya cambios en las asignaciones.

In [ ]:
from scipy.cluster.vq import kmeans, vq

In [ ]:
# Centroides, inercia...
centroids, avg_distance = kmeans(housing['medianIncome'], 5)
groups, cdist = vq(housing['medianIncome'], centroids)

In [ ]:
# Para elegir k de manera algo más informada, ilustramos 
# el ejemplo con medianIncome, cuyos estratos ya hemos analizado
# anteriormente.
y = np.arange(0,housing['medianIncome'].shape[0])
plt.scatter(housing['medianIncome'],  y , c=groups)
plt.xlabel('Salaries')
plt.ylabel('Indices')
plt.show()

A la hora de determinar el número óptimo de clusters existen diversas estrategias, una de las más populares (ilustrada abajo) es la regla del codo (elbow rule). Otro método ampliamente utilizado es el Silhouette analysis que basicamente asigna una puntuación (entre -1 y 1) a cada observación dependiendo de:
* La distancia de dicha observación al centro de su propio cluster (`a`).
* La distancia de dicha observación al cluster más cercano (`b`).

Silhouette coefficient para una muestra = `(b - a) / max(a, b)`

Vamos a echar un vistazo a la regla del codo

In [ ]:
# Definimos el número de clusters con el que queremos probar
nClusters = np.arange(1,13)
distances = []

In [ ]:
# Calculamos kmeans para cada una de las posibles configuraciones
for nc in nClusters:
    centroids, avg_distance = kmeans(housing['medianIncome'], nc)
    distances += [avg_distance]

In [ ]:
# Una vez tenemos la distancia media a los centroides para
# cada una de las configuraciones, pintamos el conjunto.
# La clave está donde se forma el codo.
plt.plot(nClusters, distances)
plt.xticks(nClusters)
plt.title("Elbow rule")
plt.xlabel("Number of clusters")
plt.ylabel("Avg. distance")
plt.show()

De nuevo: no es algo trivial la detección de outliers.

##### Aproximación gráfica

Se trata de recurrir a distintas visualizaciones para comprobar si pueden existir o no outliers evidentes.

In [ ]:
fig = plt.hist(housing["medianIncome"], bins=50)
plt.show()

Histogramas, gráficos de dispersión, de bigotes...con `matplotlib` y `seaborn`, se pueden abarcar todas estas posibilidades.

#### Preprocesamiento de outliers
Algunas posibilidades para preprocesar outliers:

##### Imputación de valores missing al uso
Si se tiene la certeza de que los outliers detectados, se deben a algún error durante el proceso de recogida, a efectos prácticos se pueden considerar valores *missing* y se procederá a su imputación como si de tales se trataran (imputando la media, la mediana, la moda...)

##### Trimming
Eliminar aquellos outliers directamente del dataset. Logicamente este método no es viable si el número de outliers es alto. La técnica para realizar esto es la que se utilizó anteriormente cuando se planteó el problema del techo en los valores del label medianHouseValue:

In [ ]:
# Capturamos aquellas observaciones cuyo medianHouseValue sea superior a 500000
index = housing[housing['medianHouseValue'] >= 500000].index

In [ ]:
# SANITY CHECK: Comprobamos que al hacer el drop de dichas observaciones se consigue el objetivo deseado
assert (housing.drop(index)["medianHouseValue"] >= 500000).value_counts().get(True, None) is None, \
"Sigue habiendo distritos con medianHouseValue mayor o igual que 50000"

In [ ]:
# Todo bien, ahora ya se puede ejecutar
housing.drop(index, inplace=True)

##### Top/Bottom/Zeroing
 * Si se sabe que para una determinada feature existe un valor que no puede ser excedido. Lo que se puede hacer es: primero identificar las observaciones que exceden ese valor y luego asignarlas ese tope que no puede ser excedido.
 * Análogamente, *Bottom* se aplicará al lado izquierdo de la distribución: aquellos valores por debajo de un determinado umbral serán configurados a ese umbral.
 * *zeroing* es un caso similar a los anteriores que se aplica para el caso de features que no pueden tomar valores negativos. De este modo serán considerados outliers aquellos valores que sí que tomen valores negativos y se procesarán asignándoles cero como valor.
 
Veamos como abordar el primer caso (el resto serían muy similares):
 * Por ejemplo se desea que la feature medianIncome no sea mayor de 13 en ningún caso. 

In [ ]:
# Por curiosidad...¿cuántos distritos cumplen con la condición?
wealthyBlocks = len(housing[housing["medianIncome"] > 13])

# Seguido se cacula la proporción
wealthyBlocks/len(housing["medianIncome"])

¡Muy poquitos!

In [ ]:
# Se sustituye por el valor deseado
housing.loc[housing["medianIncome"] > 13, "medianIncome"] = 13

In [ ]:
# Se comprueba que ha funcionado, ahora 13 es el tope de medianIncome
housing["medianIncome"].max()

##### Discretización

Se trata de un proceso en el cual una feature continua se transforma en una feature discreta de forma que sus valores se reparten en un conjunto de intervalos que cubre todos los valores de dicha feature.

### Correlaciones

Puesto que nuestro dataset no es muy grande, se puede calcular de forma poco costosa el coeficiente de correlación de Pearson (Rxy) que nos permite descubrir relaciones **lineales** de dependencia entre variables independientemente de la escala en la que se midan estas.
* Si Rxy = 1, existe una correlación positiva perfecta. Existe una dependencia total entre las dos variables (denominada relación directa) de modo que si una de ellas aumenta, la otra también lo hace en una proporción constante.
* Si 0 < Rxy < 1, existe una correlación positiva.
* Si Rxy = 0, no existe relación lineal, aunque esto no quiere decir que ambas variables sean independientes, pueden existir aun relaciones no lineales entre ellas.
* Si Rxy -1 <  Rxy < 0, existe una correlación negativa.
* Si Rxy = -1, existe una correlación negativa perfecta. Existe una dependencia total entre la dos variables (denominada relación inversa) de modo que si una de ellas aumenta, la otra disminuye en proporción


En la explicación anterior se ha señalado en negrita la palabra **lineales** y es que el coeficiente de correlación se centra en este tipo de relaciones que consisten por ejemplo en,
* Si el valor una variable X aumenta, el valor de otra variable Y lo hace también o por contra disminuye. 

Pero existen otro tipo de relaciones **no lineales** que este coeficiente no recoge, 
* Supongamos por ejemplo que si el valor de la variable X es próximo a 0.5, el valor de Y disminuye.

A continuación algunos ejemplos de correlaciones (imagen obtenida de [Wikipedia](https://en.wikipedia.org/wiki/Correlation_and_dependence#/media/File:Correlation_examples2.svg))
* En la última fila apreciamos que, aunque existe algún tipo de independencia, esta no puede ser explicada mediante relaciones lineales.
* En la segunda línea la conclusión que podemos sacar es que la fuerza la correlación no tiene que ver necesariamente con lo pronunciado de la pendiente en la gráfica resultante
<img src="images/correlation.png">

In [ ]:
# Restauremos el DataFrame para resturar potenciales outliers
# que hayamos quitado y así poder seguir explorando posibles anomalías
housing = strat_train_set.copy()

In [ ]:
corr_matrix = housing.corr()

In [ ]:
corr_matrix

Con la instrucción que acabamos de ejecutar, ya tendríamos las correlaciones. Si queremos ponerlas bonitas, tendremos que dedicarle tiempo y probar las distintas opciones que nos ofrece `matplotlib`

In [ ]:
# Indicamos las dimensiones de la figura
fig = plt.figure(1, figsize=(6,6))
# Puesto que solo queremos que tenga un subplot lo indicamos mediante 111
ax = fig.add_subplot(111)
labels = ['labels']+corr_matrix.columns.tolist()
ax.set_xticklabels(labels, rotation='45', ha='left')
ax.set_yticklabels(labels, rotation='horizontal', ha='right')

corr_mat_plot = ax.matshow(corr_matrix, cmap=plt.cm.hot_r)
# Con esto indicamos explicitamente que el rango de nuestros valores será -1,1
corr_mat_plot.set_clim([-1,1])
cb = fig.colorbar(corr_mat_plot)
cb.set_label("Correlation Coefficient")

plt.show()

Aunque a veces ["seaborn"](https://seaborn.pydata.org/introduction.html) nos hace la vida un poco más fácil:

In [ ]:
sns.heatmap(corr_matrix, 
            annot=True,
            cbar_kws={"label": "Correlation coefficient", "shrink": 1.25, "ticks": np.linspace(-1.0, 1.0, 9)}, 
            cmap=plt.cm.hot_r, 
            vmin=-1, vmax=1)
plt.show()

Y si queréis algo más interactivo, `hvplot` puede ayudar:

In [ ]:
corr_matrix.hvplot.heatmap(flip_yaxis=True, rot=45, cmap=plt.cm.hot_r)

Obviando la diagonal de la matriz (que relaciona cada feature consigo misma) tenemos,
* Una muy fuerte **relación directa** entre **totalRooms** y **totalBedRooms**, lo cual es razonable (cuanto más habitaciones haya en un distrito más probable es que haya más dormitorios)

* También se ve una muy fuerte **relación directa** entre **population**, **totalRooms** y **households** (lo cual también parece razonable)

* Vemos algo que parece más interesante que lo anterior y es que parece que existe importante **relación directa** entre la renta media (**medianIncome**) y nuestra variable target **medianHouseValue**.

In [ ]:
corr_matrix["medianHouseValue"].sort_values(ascending=False)

Otra de las funcionalidades que `pandas` ofrece apoyándose en `matplotlib` es `scatter_matrix()`, que nos dibuja una matriz en la que cada elemento es un gráfico de dispersión entre cada feature que reciba como parámetro, solo hay que imaginarse una matriz similar a la anterior reemplazando cada pequeña celda coloreada por un gráfico de dispersión, sería grande ¿verdad? Por eso solo vamos a pintarlo para las tres variables que parecen algo más correladas con medianHouseValue.

In [ ]:
from pandas.plotting import scatter_matrix

In [ ]:
attributes = ["medianHouseValue", "medianIncome", "totalRooms", "housingMedianAge"]
scatter_matrix(housing[attributes], figsize=(12,8))
plt.show()

Vemos que, como la matriz de correlaciones ya apuntaba, la relación aparentemente más prometedora es entre medianIncome y medianHouseValue. Todo un detalle por parte de `pandas` es que en la diagonal principal, en vez de mostrarnos el gráfico de dispersión de una variable contra si misma, nos muestra el histograma de esa variable (bastante más informativo).
Vamos a hacer zoom sobre esta posible relación,

In [ ]:
housing.plot(kind="scatter", x="medianIncome", y="medianHouseValue", alpha=0.5)
plt.show()

Es interesante este zoom ya que se aprecian tres líneas algo sospechosas,
* La que ya conocíamos que situaba un techo de 500000 USD para medianHouseValue
* Una segunda que parece asomarse alrededor de los 450000 USD
* Una tercera que se aprecia claramente sobre los 350000 USD

Esto es algo de lo que se podría informar al equipo de recolección de datos, por si tuvieran alguna explicación; podría ser recomendable intentar quitar esos distritos de nuestro set para evitar que nuestro algoritmo aprenda de estos patrones extraños.

### Combinando features

De momento hemos descubierto algunas cosas que pudieran ser interesantes, como por ejemplo,
* *Algunos patrones extraños en los datos* que sería interesante investigar (como las líneas en el gráfico de dispersión de *medianHouseValue vs medianIncome*)
* *Distribuciones* de nuestras features bastante *asimétricas hacia la derecha*.
* **Correlaciones interesantes** entre algunas de nuestras features.

Precísamente en este último apartado parece que nos permitió ver que entre las features totalRooms, totalBedrooms, households y population existía una relación directa ¿qué ocurre si intentamos combinar algunas de estas features para, con un poco de suerte, obtener otra más explicativa de la variable target?

In [ ]:
# roomsPerHousehold = totalRooms / householdsweighed_medianHouseValue
housing["roomsPerHousehold"] = housing["totalRooms"] / housing["households"]

In [ ]:
# bedroomsPerHousehold = totalBedrooms / households
housing["bedroomsPerHousehold"] = housing["totalBedrooms"] / housing["households"]

In [ ]:
# populationPerHousehold = population / households
housing["populationPerHousehold"] = housing["population"] / housing["households"]

In [ ]:
# bedroomsPerRoom = totalBedrooms / totalRooms
housing["bedroomsPerRoom"] = housing["totalBedrooms"] / housing["totalRooms"]

Vamos a ver si la matriz de correlación arroja algo nuevo

In [ ]:
corr_matrix = housing.corr()

In [ ]:
corr_matrix["medianHouseValue"].sort_values(ascending=False)

¡Bueno! roomsPerHousehold aporta cierta información indicándonos que, como parece lógico, cuanto más habitaciones tiene una casa en teoría más grande será y por tanto más cara.
Por otro lado parece que cuanto más habitaciones de una casa están dedicadas a dormitorios, menor es el valor de esta.
De todos modos tampoco hay que ser desde el primer momento completamente obsesivo por probar todas las combinaciones posibles, **el objetivo es conseguir conocimiento sobre los datos, montar un prototipo, analizar su salida e ir iterando de esta manera**

### Preparación de los datos

Es hora de adaptar nuestro dataset para ponerle las cosas "fáciles" a nuestro modelo, es decir, disponer los datos de una manera que *un algoritmo de Machine Learning* se sienta cómodo con ellos. A la hora de hacer esto es conveniente, en vez de hacerlo manualmente, **invertir tiempo en montar funciones por las siguientes razones**,
* Nos permitirán **reutilizar nuestras transformaciones** en cualquier futuro dataset que nos pasen.
* Incluso para futuros proyectos, ir **preparando una librería con nuestras funciones** será de gran utilidad.
* Además podremos usar estas funciones en nuestro sistema productivo para **integrarlas en el pipeline** que permitirá procesar estos.
* El proceso de probar varias transformaciones y finalmente quedarnos con aquellas que nos resulten más útiles será más fácil y rápido cuantas menos manualidades hagamos.

Primeramente obtengamos de nuevos nuestros datos limpios y separemos las features de la variable target.

In [ ]:
housing = strat_train_set.drop("medianHouseValue", axis=1)
housing_labels = strat_train_set["medianHouseValue"].copy()

In [ ]:
housing.head()

*Poner las cosas fáciles* puede sonar ambiguo y es que cada modelo es un mundo de manera que las funciones de pérdida (loss functions) que cada uno tiene como objetivo minimizar, son distintas (con el enfoque común de que el objetivo es cometer el menor error posible de acuerdo a ellas). De este modo, no todos los modelos se benefician por igual de todas las técnicas. Una de las más habituales como es la estandarización de features es conveniente en el caso de las regresiones (lineal y logística), k nearest neighbours (KNN), Support Vector Machines (SVMs) y redes neuronales...pero en el caso de los árboles de decisión (y sus ensembles) o modelos bayesianos, no se suele aplicar.

#### Valores missing, nulos, NaN...

Son pocos los modelos que se sienten confortables con **valores ausentes (null, NaN, blancos...)**; como recordaremos, al principio introdujimos adrede unos cuantos NaN en totalBedrooms y ahora toca encargarse de ellos. Para ello tenemos principalmente tres opciones,
* **Eliminar** los **distritos** que contengan algún NaN, en este caso para esta feature.
* **Eliminar** la **feature** directamente, sin contemplaciones.
* "Rellenar" esos valores ausentes con algún valor que creamos que pueda ser adecuado (media, mediana, cero...). Este concepto se conoce como **Imputación**

Lo bueno es que `pandas` pone a nuestro alcance poderosos métodos para el tratamiento de este tipo de valores en nuestros DataFrame. Para la **primera opción**: `dropna`

In [ ]:
housing.head()

In [ ]:
housing.dropna(subset=["totalBedrooms"]).head()

Para la **segunda opción**: `drop`

In [ ]:
housing.drop("totalBedrooms", axis=1).head()

Para la **tercera opción** (si decidieramos por ejemplo rellenar los NaN con la mediana): `fillna` con el valor que consideremos

In [ ]:
median = housing["totalBedrooms"].median()
housing["totalBedrooms"].fillna(median).head(10)

Si optamos por la opción tres, guardaremos el valor calculado de la mediana (en este caso) como oro en paño, ya que este será el que tengamos que utilizar también para reemplazar aquellos NaN de totalBedrooms en el conjunto de test y también cuando nuestro modelo productivo empiece a recibir datos.....

<img src="images/computerguy.png">

En este punto, **¡comprueba tu versión de Scikit-learn!**, a partir de la versión 0.20.0 se han introducido cambios relevantes en los módulos de preprocesamiento de la librería.

In [ ]:
import sklearn

In [ ]:
print("Scikit-learn version " + sklearn.__version__)

* **SimpleImputer**, `scikit-learn` le da una vuelta de tuerca al antiguo `Imputer` dando como resultado el `SimpleImputer`. Entre las mejoras realizadas tenemos,
    * Posibilidad de reemplazar valores categóricos.
    * Imputación de un valor constante (fill value)
    
  Más información [aquí](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html).

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imputer = SimpleImputer(strategy="median")

Tanto `Imputer` (en su momento) como `SimpleImputer`, actuarán sobre todas las features del DataFrame desde el que se invoque. Esto quiere decir que la estrategia indicada a través del parámetro `strategy` se aplicará a todas las features por igual.
Puesto que se ha elegido aplicar la estrategia `median` (mediana), que es una estrategia aplicable de forma exclusiva a features numéricas, crearemos una copia de nuestros datos excluyendo aquellas features no numéricas (ocean_proximity).

In [ ]:
housing_num = housing.drop("ocean_proximity", axis=1)

Ahora ya podemos entrenar nuestro `SimpleImputer` sobre el dataset que contiene exclusivamente features numéricas, esto lo haremos mediante el método `fit`, este método lo que hace basicamente es "entrenar" nuestro `SimpleImputer` antes de aplicarlo a un dataset mediante el método `transform`.

In [ ]:
imputer.fit(housing_num)

In [ ]:
imputer.statistics_

In [ ]:
# Si se utiliza notación científica por defecto, es posible cambiar este comportamiento
# Si modificamos el parámetro suppress del método set_printoptions de numpy a True
# siempre se imprimirán los números en coma flotante en notación decimal
np.set_printoptions(suppress=True)

In [ ]:
housing_num.median().values

Vemos que el `Imputer`/`SimpleImputer` obtiene los mismos resultados que si aplicáramos la mediana "a pelo". La gran ventaja del `Imputer`/`SimpleImputer` es que nos cubre en caso de que ya sea en el conjunto de test o más adelante, alguna de nuestras features numéricas tenga algún NaN; él se encargará de tomar todos los NaN sean de la feature numérica que sean y reemplazarlos por la mediana calculada (o lo que sea, según la estrategia configurada)

Ahora utilizaremos el `Imputer`/`SimpleImputer` entrenado para reemplazar los NaN existentes por las medianas aprendidas

In [ ]:
X = imputer.transform(housing_num)

In [ ]:
X

No es complicado reconventir este array de `numpy` a un DataFrame de `pandas`

In [ ]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns)

In [ ]:
housing_tr.info()

`scikit-learn` proporciona [otras](https://scikit-learn.org/stable/modules/impute.html) clases que plantean métodos de imputación más sofisticados. Aunque al final, el más adecuado lo determinará seguramente nuestro caso de uso.

<h3><center>Scikit-Learn: sólidos principios de diseño.</center></h3>
<center>API design for machine learning software: experiences from the scikit-learn project (https://arxiv.org/pdf/1309.0238v1.pdf)<center>

* **Consistencia**, todos los objetos comparten una API simple y consistente.

    a) Estimators, cualquier objeto que pueda estimar parámetros en base al dataset es considerado un Estimator (por ejemplo un `SimpleImputer`). La estimación se realiza mediante el método `fit()`, que toma como parámetro el dataset (también puede admitir dos parámetros siendo el segundo la parte del dataset que contiene las labels)
    
    b) Transformers, algunos Estimators (como el propio `SimpleImputer`), pueden también transformar un dataset. Estos son los Transformers y cuentan con un método llamado `transform()` que se encarga de transformar, en base a los parámetros aprendidos, el dataset que recibe por parámetro devolviéndolo transformado. Cuentan además con un método llamado `fit_transform()` (cuyos resultados son similares a llamar a `fit()` y a `transform()` de manera consecutiva, pero a veces está optimizado para mejorar tiempos de ejecución)
    
    c) Predictors, existen Estimators que pueden hacer predicciones dado un determinado dataset. Por ejemplo el modelo `LinearRegression` es un Predictor y tiene un método `predict()` que le permite devolver las predicciones para un dataset nuevo que reciba. Además cuenta con un método `score()` que evalua la calidad de las predicciones realizadas comparándolas con el conjunto de etiquetas (en el caso del aprendizaje supervisado).
    

* **Inspección**, todos los hiperparámetros de un estimador son accesibles mediante variables de la propia instancia (en el caso de `SimpleImputer`, podríamos acceder a la estrategia configurada mediante `SimpleImputer.strategy`) incluso los parámetros aprendidos pueden ser también accedidos como vimos con `SimpleImputer.statistics_` (este tipo de propiedades se caracterizan porque llevan como sufijo un underscore)

* **No proliferación de clases**, se respeta la máxima de no reinventar la rueda: los datasets se modelan como DataFrames de `pandas`, arrays de `numpy` o matrices dispersas de `scipy`. En cuanto a los hiperparámetros son strings o números de Python normales y corrientes.

* **Composition**, las piezas ofrecidas por `scikit-learn`, se pueden utilizar como bloques de construcción de Pipelines, como se verá más adelante en el curso. ¿Parece interesante? ¿quieres saber más? echa un vistazo a la diapositiva *00_project_Flow.ipynb (Composition)*.

* **Defaults razonables**, los valores por defecto configurados para los múltiples parámetros son razonables, es decir, permiten crear un sistema funcional del cual partir, sin grandes quebraderos de cabeza.


#### Gestionando texto y features categóricas (nominales y ordinales)

El hecho de que a lo largo de estos pasos hayamos estado marginando el campo **ocean_proximity**, viene dado porque al ser un **atributo no numérico**, hay pocos cálculos que podamos hacer con el (por ejemplo no podríamos calcular la mediana). Además la mayoría de **los algoritmos de Machine Learning trabajan mejor con números que con categorías**. Una clase que nos puede ayudar a la hora de trabajar con features categóricas es,
   * **OrdinalEncoder**, recibirá una o más features de tipo entero o de tipo categórico y devolverá una columna de números (su tipo vendrá indicado por el parámetro `dtype`) por cada feature, cuyos valores irán desde *0* hasta el *número de categorías - 1*. Veamos un ejemplo:



In [ ]:
team = pd.DataFrame({'position':['keeper','defender','defender','defender','midfielder','forward'], 
                     'seasons':[1,2,2,3,1,2]})

In [ ]:
team

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# El parámetro dtype determina el tipo de dato de las 
# categorías devueltas por el OrdinalEncoder.
# Ha de ser numérico (algún subtipo de entero o float)
ordEncoder = OrdinalEncoder(dtype=np.uint8)

In [ ]:
ordEncoder.fit(team)

In [ ]:
# El atributo categories_ identifica las categorías 
# identificadas por el OrdinalEncoder para cada feature
ordEncoder.categories_

In [ ]:
ordEncoder.transform(team)

¡Cuidado!

In [ ]:
signing = pd.DataFrame({'position':['manager'], 'seasons':[1]})

In [ ]:
# Antes de ejecutar...¿Qué crees que ocurrirá? ¿Cómo se comportará el OrdinalEncoder?
ordEncoder.transform(signing)

Más recomendable:

In [ ]:
# El parámetro dtype determina el tipo de dato de las 
# categorías devueltas por el OrdinalEncoder.
# Ha de ser numérico (algún subtipo de entero o float)
ordEncoder = OrdinalEncoder(categories=[['keeper','defender','midfielder','forward','manager'], 
                                        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]], dtype=np.uint8)

In [ ]:
signing

In [ ]:
ordEncoder.fit_transform(signing)

In [ ]:
ordEncoder.categories_

`OrdinalEncoder` permite a través de su propiedad `categories_` listar explícitamente las categorías de nuestras features, eso es de gran ayuda para asegurarnos de que todas las categorías que nos interesan serán contempladas por nuestro transformer ([disponible](https://github.com/scikit-learn/scikit-learn/pull/12367) a partir de la versión 0.20.1 y posteriores).

Apliquemos el `OrdinalEncoder` a nuestra feature categórica ocean_proximity:

In [ ]:
housing_cat = housing['ocean_proximity'].copy()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

housingOrdEncoder = OrdinalEncoder(dtype=np.uint8)
housing_cat_encoded = housingOrdEncoder.fit_transform(housing_cat.values.reshape(-1,1))

In [ ]:
housingOrdEncoder.categories_

In [ ]:
housing_cat_encoded

* **LabelEncoder**, se trata de un transformer que se ideó para codificar el target o label en problemas de clasificación. A cada posible clase (representada por un entero o un string) le asignará un entero entre 0 y el *número de clases - 1*.

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
housing_cat.shape

In [ ]:
le.fit(housing_cat)

In [ ]:
le.transform(housing_cat)

In [ ]:
le.classes_

El problema con este tipo de representación es que existen algoritmos de Machine Learning que pueden asumir que dos categorías asociadas a valores próximos (0 y 1) son más similares entre si que dos categorías asociadas a valores más lejanos (0 y 4) y en nuestro caso esto no nos interesa en absoluto. 
Para resolver este problema podemos recurrir a la notación **One-Hot Encoding** que consiste en representar cada categoría en una combinación de bits tales que todos serán 0 salvo uno de ellos que será 1 (alternando este entre los distintos valores que componen la categoría).¿One hot Chili Peppers? echa un vistazo a la diapositiva *00_project_Flow.ipynb (One-hot)*.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
encoder = OneHotEncoder()
housing_cat_1hot = encoder.fit_transform(housing_cat.values.reshape(-1,1))
housing_cat_1hot

El resultado es una matriz dispersa (`sparse matrix`) de `scipy`: las matrices dispersas son muy interesantes para aquellas situaciones en las que tenemos una matriz en la que la mayor parte de la información son ceros. En este caso tenemos cerca de 17000 filas de las cuales tan solo una columna contiene un 1 por fila; sería un desperdicio de memoria utilizar un montón de espacio en almacenar ceros, la matriz dispersa soluciona esto almacenando las posiciones de los elementos relevantes (los que no son cero). Se puede convertir en array de `numpy` fácilmente.

In [ ]:
encoder.categories_

In [ ]:
housing_cat_1hot.toarray()

In [ ]:
housing_cat

* **LabelBinarizer**, como su propio nombre indica permite binarizar los valores de nuestro target o label. En el caso de que solo pueda tomar dos posibles valores (sano/enfermo, ok/ko, sobrevive/fallece...) la binarización convertirá los valores en 1 y 0. Si por el contrario tenemos un problema multiclase (pobre/medio/rico, muy barato/barato/en la media/caro/muy caro...) `LabelBinarizer` transformará los posibles valores de la clase a notación one-hot. El resultado es un array de `numpy`, aunque esto es configurable mediante un parámetro: `sparse_output`, que si se configura a `True`, devolverá una sparse matrix de `scipy`. 

In [ ]:
from sklearn.preprocessing import LabelBinarizer

In [ ]:
encoder = LabelBinarizer()
housing_cat_1hot = encoder.fit_transform(housing_cat)
housing_cat_1hot

In [ ]:
encoder.classes_

#### ColumnTransformer

`Scikit-learn` ya ofrecía una amplia cantidad de transformers que permiten llevar a cabo distintas operaciones sobre nuestros datos. El problema es que, en el mundo real, es sencillo encontrarse datos en los que las distintas features son de diversos tipos, esto implica que los requisitios de preprocesamiento no serán los mismos para todas ellas siendo necesario por tanto aplicar transformaciones de manera focalizada según el tipo de cada feature.
Hasta ahora `scikit-learn` no ofrecía una manera simple (out of the box) para esta tarea...hasta la versión 0.20.0 que incluye el `ColumnTransformer`.
Veamos un ejemplo:

In [ ]:
# DataFrame de juguete, con columnas numéricas y categóricas. En la columna edad hay un NaN
equipo = pd.DataFrame({
    "nombre": np.array(["Julio", "Nuria", "Jose", "Luis", "Daniel", "Javier", "Alberto", "Lourdes"]),
    "edad": np.array(  [     22,      26,     28,     25,       np.nan,       24,        24,        26]),
    "sexo": np.array(  ["Hombre", "Mujer","Hombre","Hombre","Hombre","Hombre", "Hombre",  "Mujer" ]),
    "demarcacion": np.array( ["portero", "defensa", "defensa", "medio", "medio", "medio", "delantero", "defensa"]),
    "estatura": np.array( [190, 187, 183, 170, 168, 180, 191, 175])
}, columns=["nombre", "edad", "sexo", "demarcacion", "estatura"])

In [ ]:
equipo

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [ ]:
ct = ColumnTransformer([("one_hot", OneHotEncoder(sparse=False, dtype=np.uint8), [2, 3]),
                        ("imputa_guay", SimpleImputer(strategy="mean"), [1])], remainder='passthrough')

In [ ]:
X = ct.fit_transform(equipo)

In [ ]:
dfX = pd.DataFrame(X)

In [ ]:
dfX

Y en el caso de que nos dé igual el poder nombrar cada step del `ColumnTransformer`, podremos recurrir a este atajo:

In [ ]:
from sklearn.compose import make_column_transformer

In [ ]:
ct_made = make_column_transformer((OneHotEncoder(sparse=False, dtype=np.uint8), [2, 3]),
                                   (SimpleImputer(strategy="constant", fill_value=30), [1]), remainder='passthrough')

In [ ]:
ct_made.fit_transform(equipo)

In [ ]:
ct_made.named_transformers_

#### Transformers personalizados

Si bien es cierto que `scikit-Learn` ofrece una gran variedad de transformers, al final llegará el momento en el que tengamos que definir los nuestros propios y queremos que al definirlos, podamos trabajar con ellos de igual manera que lo haríamos con cualquiera de los transformers out of the box (por ejemplo que podamos integrarlo en un pipeline sin ningún problema).

Se recomienda utilizar `BaseEstimator` y `TransformerMixing` como clases base, de este modo solo tendremos que implementar dos métodos para nuestra clase transformer: `fit()` y `transform()`. Por ejemplo

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
# Representamos mediante una clase nuestro transformer cuyo objetivo será realizar la combinación
# de features que anteriormente abordamos de forma manual. Además le añadimos un hiperparámetro para 
# poder elegir si se desea combinar o no las features totalBedrooms y totalRooms.

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, index_list, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
        self.indices = index_list
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        if isinstance(X, pd.DataFrame):
            X = X.values
            
        room_ix, bedroom_ix, population_ix, household_ix = self.indices
        
        rooms_per_household = X[:, room_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedroom_ix] / X[:, room_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]
        
        

In [ ]:
# Declaramos el parámetro que hemos establecido
# como obligatorio para nuestro constructor
featEng_indices = [3,4,5,6]

In [ ]:
# Invocamos al constructor como cualquier Transformer out of the box
caa = CombinedAttributesAdder(featEng_indices, add_bedrooms_per_room=False)

In [ ]:
# Con el objeto creado invocamos fit_transform()
housFeatEng = caa.fit_transform(housing)

In [ ]:
# Reconvertimos la matriz de numpy a un dataframe para
# poder comprobar los resultados
pd.DataFrame(housFeatEng, columns=list(housing) + ['rooms_per_household','population_per_household']).head()

La inclusión de hiperparámetros (¡sí! `add_bedrooms_per_room` es un hiperparámetro) en nuestros transformadores custom permite habilitar o deshabilitar de forma sencilla aquellos pasos de preparación de datos sobre cuya efectividad no estamos muy seguros; esto permite probar distintas combinaciones de forma fácil y rápida.

In [ ]:
# Como ya hemos podido comprobar, el procesamiento de features numéricas
# y de features categóricas, son diferentes. Es por ello muy posible que
# queramos que la parte categórica de nuestro dataframe y la parte numérica
# sigan procesamientos diferentes (pipelines diferentes)
# Hasta la llegada del ColumnTransformer no había una solución out of the box
# para conseguir esto. Sin embargo podíamos hacer la nuestra propia de forma sencilla
# Creemos un sencillo transformer que nos ayude

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, feature_names):
        self.feature_names = feature_names
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.feature_names].values

In [ ]:
cols=['totalBedrooms','medianIncome']

In [ ]:
df1 = DataFrameSelector(cols)

In [ ]:
res = df1.fit_transform(housing)

In [ ]:
res

#### Escalado de features

Esta es una parte muy a tener en cuenta ya que los algoritmos de Machine Learning se desenvolverán mejor o peor dependiendo de la manera en la que reciban los datos y un punto que afecta a muchos de ellos es el recibir las features numéricas en escalas radicalmente diferentes entre si, por ejemplo: **totalRooms oscila entre 2 y 39320 mientras que medianIncome va de 0 a 15**. Existen dos formas principales de gestionar este tema,
* **Min-Max Scaling**, se trata de ajustar los valores de cada feature de manera que caiga dentro del rango de valores que hemos definido (por defecto entre 0 y 1), esto se consigue aplicando la siguiente operación

                        X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
                        X_scaled = X_std * (max - min) + min

Desde `Scikit-Learn` se proporciona la clase `MinMaxScaler`, que se encarga de realizar este tipo de escalado sobre cada feature de manera que sus valores se ciñan al rango deseado (el cual es además configurable a través del hiperparámetro `feature_range`)

* **Standardization**, responde a la predilección que sienten los algoritmos de Machine Learning por aquellos datos distribuídos normalmente (media 0 varianza 1); basicamente para cada feature calcula la diferencia entre cada valor y la media y divide el resultado entre la varianza. El punto fuerte de esta técnica es que, al no tener un máximo y un mínimo establecidos, es menos sensible a outliers: si por ejemplo tuvieramos un medianIncome que en vez de estar en el rango 0-15 se fuera hasta 100, un `MinMaxScaler` apelotonaría todos los valores de medianIncome entre 0 y 0.15 siendo el 1 el outlier con 100.

A través del `StandardScaler`, `Scikit-learn` implementa esta técnica.
<img src="images/standardization.png">

Ten en cuenta dos detalles importantes, 
* El escalado se aplica basicamente a las features; rara vez es requerido en la variable target.
* Los Scaler requieren previamente a la invocación de su método `transform()`, que se aplique su método `fit()` sólamente sobre el conjunto de datos de training (lo de siempre, el conjunto de test, ni tocarlo), una vez hecho esto ya se podrá aplicar el método `transform()` sobre el propio conjunto de training, el de test y los nuevos datos.
* Al igual que con los métodos de imputación, no solo existen los aquí citados. Podemos ver más de los disponibles en `Scikit-learn` [aquí](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-scaler)

# BONUS
¡Atención! por haber llegado hasta aquí, os merecéis un bonus.

#### Pandas Profiling
Habéis descubierto una librería que lleva varios niveles más allá la información dada por el método `describe()` de `pandas`. Genera un informe en HTML y CSS3 que aporta detallada información estadística y gráfica para cada variable.
* Para instalar `pandas profiling` podemos hacerlo mediante
    * `conda install pandas-profiling` si queremos usar el conda package manager.
    * `pip install pandas-profiling` si queremos usar el pip package manager.
* Una vez instalado podremos importarlo en nuestro notebook mediante `import pandas_profiling`
* Para poder generar un reporte con `pandas profiling`, tendremos que haber cargado primeramente nuestros datos
en un DataFrame de `pandas`.
* Una vez tenemos nuestros datos en un DataFrame ejecutaremos `pandas_profiling.ProfileReport(df)` y el reporte se mostrará en nuestro notebook

Al mismo tiempo `pandas profiling` descarta por defecto aquellas features que estén áltamente correladas (hablamos de un umbral superior por defecto a 0.9) con otras features ya existentes, aunque brinda opciones para desactivar el chequeo de correlaciones, cambiar el umbral e incluso proporcionar una lista de variables que no serán rechazadas aunque no superen el chequeo de correlación.

Como no, es posible además guardar en disco el reporte generado, para ello recogeremos el resultado de ejecutar el ProfileReport: 
`pR = pandas_profiling.ProfileReport(df)` y seguídamente lo almacenaremos en la ruta deseada `pR.to_file(outputfile="myoutputfile.html")`